In [5]:
import pandas as pd
import time
import plotly.graph_objects as go 
import plotly.express as px
import numpy as np

def JsonParser(data):
    import json
    js = json.loads(data)
    return js


df_list = pd.read_csv('listings.csv', converters={'amenities':JsonParser})

df_compare = pd.DataFrame(columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE'])

C:\Users\manar\AppData\Local\Temp\ipykernel_22372\2050797132.py:13: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = pd.read_csv('listings.csv', converters={'amenities':JsonParser})


In [49]:
df_list.shape

(39881, 75)

In [50]:
#df_calendar = df_calendar.drop(columns=['adjusted_price','minimum_nights','maximum_nights'])

In [51]:
df_list['source'].unique()

array(['city scrape', 'previous scrape'], dtype=object)

In [52]:
df_list['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Hotel room', 'Shared room'],
      dtype=object)

In [53]:
df_list['price']

0        $299.00
1        $175.00
2         $60.00
3        $425.00
4        $275.00
          ...   
39876    $157.00
39877    $108.00
39878    $140.00
39879    $400.00
39880     $40.00
Name: price, Length: 39881, dtype: object

In [55]:
df_list.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [56]:
columns_not_incluted = ['id','listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified','latitude', 'longitude',
       'calendar_last_scraped','first_review',
       'last_review','license', 
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'
       ]

In [57]:
df_list = df_list.drop(columns_not_incluted, axis=1)
df_list.head(5)

,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,...,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,"Brooklyn , New York, United States",Kensington,Brooklyn,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,1.0,...,0,4.89,4.88,5.00,5.00,5.00,4.75,4.88,f,0.11
1,"New York, United States",Midtown,Manhattan,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,...,0,4.68,4.73,4.63,4.77,4.80,4.81,4.40,f,0.31
2,NaN,Bedford-Stuyvesant,Brooklyn,Private room in rental unit,Private room,2,NaN,NaN,1.0,1.0,...,0,4.52,4.22,4.09,4.91,4.91,4.47,4.52,f,0.31
3,NaN,Ridgewood,Queens,Entire townhouse,Entire home/apt,16,NaN,2.5 baths,5.0,10.0,...,0,4.42,4.64,4.36,4.82,5.00,4.82,4.55,f,0.10
4,NaN,Sunset Park,Brooklyn,Entire rental unit,Entire home/apt,4,NaN,1.5 baths,2.0,2.0,...,1,5.00,5.00,5.00,5.00,5.00,4.67,5.00,f,0.03


In [58]:
df_list['bathrooms_text'].unique(), df_list['bathrooms_text'].isna().sum()

(array(['1 shared bath', '1 bath', nan, '2.5 baths', '1.5 baths',
        '1 private bath', '1.5 shared baths', '2 baths', '2 shared baths',
        'Shared half-bath', '3 baths', 'Half-bath', '0 baths',
        '0 shared baths', 'Private half-bath', '4 baths', '4.5 baths',
        '5.5 baths', '2.5 shared baths', '3.5 baths', '5 baths',
        '3 shared baths', '4 shared baths', '6 baths', '3.5 shared baths',
        '4.5 shared baths', '7.5 baths', '6.5 baths', '7 baths',
        '6 shared baths', '9.5 baths'], dtype=object),
 84)

In [59]:
df_list['beds'].unique()

array([ 1., 10.,  2.,  5.,  3.,  8.,  4.,  6., nan,  9.,  7., 18., 12.,
       16., 11., 21., 13., 24., 17., 14.])

In [60]:
df_list.isna().sum()[df_list.isna().sum() > 0]

neighbourhood                  16414
bathrooms                      39881
bathrooms_text                    84
bedrooms                        3783
beds                             884
calendar_updated               39881
review_scores_rating            8362
review_scores_accuracy          8825
review_scores_cleanliness       8815
review_scores_checkin           8829
review_scores_communication     8820
review_scores_location          8832
review_scores_value             8832
reviews_per_month               8362
dtype: int64

In [61]:
columns_with_nulls = ['neighbourhood' ,'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated']

In [62]:
df_list = df_list.drop(columns_with_nulls, axis=1)

In [63]:
#df_list = df_list[df_list['reviews_per_month'] > 0]
df_list = df_list.dropna()
df_list.isna().sum()[df_list.isna().sum() > 0]
df_list.shape, df_list.columns

((27578, 26),
 Index(['neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
        'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
        'has_availability', 'availability_30', 'availability_60',
        'availability_90', 'availability_365', 'number_of_reviews',
        'number_of_reviews_ltm', 'number_of_reviews_l30d',
        'review_scores_rating', 'review_scores_accuracy',
        'review_scores_cleanliness', 'review_scores_checkin',
        'review_scores_communication', 'review_scores_location',
        'review_scores_value', 'instant_bookable', 'reviews_per_month'],
       dtype='object'))

In [64]:
df_list.dtypes

neighbourhood_cleansed          object
property_type                   object
room_type                       object
accommodates                     int64
bathrooms_text                  object
bedrooms                       float64
beds                           float64
amenities                       object
price                           object
has_availability                object
availability_30                  int64
availability_60                  int64
availability_90                  int64
availability_365                 int64
number_of_reviews                int64
number_of_reviews_ltm            int64
number_of_reviews_l30d           int64
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
review_scores_value            float64
instant_bookable                object
reviews_per_month        

In [65]:
not_numeric = ['neighbourhood_cleansed', 'property_type', 'room_type', 'bathrooms_text', 'amenities', 'has_availability', 'instant_bookable']
binary_column = ['has_availability', 'instant_bookable']
not_handled = ['bathrooms_text']

In [66]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [67]:
encoder = OrdinalEncoder(categories=[['f','t']])
df_list['has_availability'] = encoder.fit_transform(df_list[['has_availability']])
df_list['instant_bookable'] = encoder.fit_transform(df_list[['instant_bookable']])
df_list['has_availability'] = df_list['has_availability'].astype('int')
df_list['instant_bookable'] = df_list['instant_bookable'].astype('int')

In [68]:
from re import sub
def remove_dollar_sign(price:str):
    return price.replace('$','').replace(',','')

def remove_quotes(value:str):
    return value.replace('"','')

def remove_weird_signs(row):
    row = row.encode('ascii', 'ignore').strip().decode('unicode_escape').replace('"','').replace('[','').replace(']','')
    row = ''.join([i if ord(i) < 128 else '' for i in row])
    row_as_list = row.split(',')
    row_as_list = [x.strip() for x in row_as_list]
    return ','.join(row_as_list)

def clear_amenities(row):
    to_clear = ['wifi','childrens','tv','sound','parking','refrigerator','dryer','washer','clothing','stove','oven','pool','gym','breakfast','garage','sauna','garden',
    'heating','hot tub','console','conditioning','chair','crib','housekeeping','free carport on premises','kitchen', 'ev charger', 'pillows', ' bar',  'fitness','restaurant']
    row = [x for x in row if x not in 'chair']
    cosmetics = ['conditioner', 'soap', 'shampoo', 'toiletries']
    sound = ['sound', 'speaker']
    row = ['cosmetics' if any(cos in x for cos in cosmetics) else x.lower() for x in row]
    row = ['sound' if any(cos in x for cos in sound) else x.lower() for x in row]
    for item in to_clear:
        row = [f'{item}' if f'{item}' in x.lower() else x.lower() for x in row]
    return row

In [69]:
df_list['price'] =  df_list['price'].apply(remove_dollar_sign).astype('float32')
#df_calendar['price'] = df_calendar['price'].apply(remove_dollar_sign).astype('float32')
df_list['amenities'] = df_list['amenities'].apply(clear_amenities)

In [70]:
df_list['property_type'].unique()

array(['Private room in rental unit', 'Entire townhouse',
       'Entire rental unit', 'Room in boutique hotel',
       'Private room in bed and breakfast', 'Private room in townhouse',
       'Private room in home', 'Private room in condo',
       'Private room in loft', 'Entire guesthouse', 'Entire guest suite',
       'Entire loft', 'Private room in guest suite',
       'Entire serviced apartment', 'Entire condo', 'Entire home',
       'Shared room in loft', 'Private room in houseboat',
       'Shared room in rental unit', 'Shared room in home',
       'Private room', 'Entire place', 'Private room in guesthouse',
       'Room in aparthotel', 'Entire cottage',
       'Private room in serviced apartment', 'Boat', 'Cave',
       'Room in serviced apartment', 'Tiny home', 'Shared room in condo',
       'Floor', 'Private room in hostel',
       'Private room in casa particular', 'Room in hotel',
       'Shared room in townhouse', 'Private room in bungalow',
       'Houseboat', 'Entire bu

In [71]:
def clean_property_type(row):
    property_type_remove = ['entire', 'private', 'room','tiny', 'in',  'shared']
    row_list = row.split()
    row_list = [x for x in row_list if x.lower() not in property_type_remove]
    return ' '.join(row_list).lower()

In [72]:
df_list['property_type'] = df_list['property_type'].apply(clean_property_type)

In [73]:
df_list['property_type'].unique()

array(['rental unit', 'townhouse', 'boutique hotel', 'bed and breakfast',
       'home', 'condo', 'loft', 'guesthouse', 'guest suite',
       'serviced apartment', 'houseboat', '', 'place', 'aparthotel',
       'cottage', 'boat', 'cave', 'floor', 'hostel', 'casa particular',
       'hotel', 'bungalow', 'resort', 'villa', 'tent', 'earthen home',
       'lighthouse', 'barn', 'vacation home', 'camper/rv', 'minsu',
       'tower'], dtype=object)

In [74]:
#df_list = df_list[df_list['beds']< 10]

In [75]:
#need to be saperate to number and isbathshared
df_list['bathrooms_text'].unique()

array(['1 shared bath', '2.5 baths', '1.5 baths', '1 private bath',
       '1 bath', '1.5 shared baths', '2 baths', '2 shared baths',
       'Shared half-bath', '3 baths', 'Half-bath', '0 shared baths',
       'Private half-bath', '4 baths', '4.5 baths', '5.5 baths',
       '2.5 shared baths', '3.5 baths', '0 baths', '3 shared baths',
       '5 baths', '4 shared baths', '6 baths', '4.5 shared baths',
       '6.5 baths', '7 baths', '6 shared baths'], dtype=object)

In [76]:
def set_bath_type(bath:str) -> str:
    if('shared'):
        return 1
    else:
        return 0

In [77]:
def set_baths_number(bath:str) -> float:
    import re
    bath = bath.lower().replace('half','0.5')
    bath = re.findall(r"[-+]?(?:\d*\.*\d+)", bath)[0]
    return float(bath)

In [78]:
df_list['bath_number'] = df_list['bathrooms_text'].apply(set_baths_number).astype('float32')
df_list['bath_is_shared'] = df_list['bathrooms_text'].apply(set_bath_type).astype('int')

In [79]:
df_list.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,has_availability,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,bath_number,bath_is_shared
0,Kensington,rental unit,Private room,2,1 shared bath,1.0,1.0,"[first aid kit, microwave, stove, coffee maker...",299.0,1,...,4.88,5.00,5.00,5.00,4.75,4.88,0,0.11,1.0,1
3,Ridgewood,townhouse,Entire home/apt,16,2.5 baths,5.0,10.0,"[dryer, essentials, carbon monoxide alarm, iro...",425.0,1,...,4.64,4.36,4.82,5.00,4.82,4.55,0,0.10,2.5,1
4,Sunset Park,rental unit,Entire home/apt,4,1.5 baths,2.0,2.0,"[dryer, tv, refrigerator, bbq grill, garden, h...",275.0,1,...,5.00,5.00,5.00,5.00,4.67,5.00,0,0.03,1.5,1
6,Greenpoint,boutique hotel,Hotel room,2,1 private bath,1.0,1.0,"[first aid kit, coffee maker, gym, long term s...",308.0,1,...,4.98,4.95,4.98,4.88,4.46,4.78,0,0.30,1.0,1
7,Midtown,rental unit,Private room,2,1 bath,1.0,1.0,"[cosmetics, microwave, laundromat nearby, mini...",68.0,1,...,4.24,3.75,4.66,4.46,4.88,4.40,0,3.39,1.0,1


In [80]:
df_list = df_list.drop(columns=not_handled)
df_list.shape

(27578, 27)

In [81]:
#df_list = df_list[df_list['room_type'] != 'Shared room']

In [82]:
df_list.to_csv('cleaned_data.csv')

In [83]:
df_list_without_dummies = df_list.copy()

In [84]:
df_list["price"].quantile(0.99)

995.0

In [85]:
df_list = df_list[df_list['availability_365'] != 365]
df_list = df_list[(df_list['price']) < (df_list["price"].quantile(0.99))]

In [86]:
amenities_dummies = pd.get_dummies(df_list['amenities'].apply(pd.Series).stack()).groupby(level=0).sum()
df_list = df_list.join(amenities_dummies)
df_list = df_list.drop(columns=['amenities'])
df_list =  pd.get_dummies(df_list)

df_list.shape

C:\Users\manar\AppData\Local\Temp\ipykernel_21800\2584880822.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  amenities_dummies = pd.get_dummies(df_list['amenities'].apply(pd.Series).stack()).groupby(level=0).sum()


(26916, 429)

In [87]:
df_list.columns

Index(['accommodates', 'bedrooms', 'beds', 'price', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews',
       ...
       'property_type_serviced apartment', 'property_type_tent',
       'property_type_tower', 'property_type_townhouse',
       'property_type_vacation home', 'property_type_villa',
       'room_type_Entire home/apt', 'room_type_Hotel room',
       'room_type_Private room', 'room_type_Shared room'],
      dtype='object', length=429)

In [88]:
df_list.shape

(26916, 429)

In [89]:
df_list = df_list.dropna()
df_list.shape

(26915, 429)

In [90]:
df_list_without_dummies.to_csv('df_without_dummies.csv')
df_list.to_csv('df_with_dummies.csv')

In [6]:
df_list = pd.read_csv('df_with_dummies.csv', index_col=0)

In [7]:
import keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split

In [8]:
df_list_Y = df_list['price']

In [9]:
df_list = df_list.drop(columns=['price'])
df_list = df_list.select_dtypes(['number'])

In [94]:
#df_list = df_list.drop(columns=['has_availability',
#       'availability_30', 'availability_60', 'availability_90',
#       'availability_365'])

In [95]:
#df_calendar['date'] = pd.to_datetime(df_calendar['date'] )
#df_calendar['weekday'] = df_calendar['date'].dt.weekday
#df_calendar['month'] = df_calendar['date'].dt.month
#df_calendar['year'] = df_calendar['date'].dt.year


In [96]:
#df_calendar.head(5)

In [97]:
#df_list = df_list.rename(index=str, columns={"id": "listing_id"})
#merged_df = pd.merge(df_list,df_calendar, on='listing_id')


In [98]:
#df_calendar.dtypes

In [99]:
#merged_df = merged_df.sort_values(by=['listing_id'])

In [100]:
#merged_df = merged_df.drop(columns=['date', 'listing_id', 'available'])
#merged_df_y = merged_df['price']
#merged_df = merged_df.drop(columns=['price'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_list, df_list_Y, test_size=0.3, random_state=42)

In [11]:
val_amount = int(df_list.shape[0]*0.1)
val_amount

2691

In [103]:
X_train.shape

(18840, 428)

In [12]:
#df_shape
val_X = X_train[0:val_amount]
val_y = y_train.iloc[0:val_amount]
X_train_part = X_train[val_amount:-1]
y_train_part = y_train.iloc[val_amount:-1]

In [105]:
from numpy import arange
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from math import sqrt

model = Lasso(alpha=0.01)

start_time = time.time()
results = model.fit(X_train, y_train)
how_long = time.time() - start_time

y_test_preds = model.predict(X_test)
mse = mean_squared_error(y_test, y_test_preds)

rmse = sqrt(mse)
result = [mse, rmse]
print(f'RMSE {rmse}')

RMSE 80.09332196918704


In [106]:
list_with_results = ['Lasso', how_long, result[0], result[1]]
df_compare = df_compare.append(pd.DataFrame([list_with_results], columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE']), ignore_index=True)

C:\Users\manar\AppData\Local\Temp\ipykernel_21800\1792374480.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_compare = df_compare.append(pd.DataFrame([list_with_results], columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE']), ignore_index=True)


In [107]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

forest = RandomForestRegressor(n_estimators=100, 
                               criterion='squared_error', 
                               random_state=42, 
                               n_jobs=-1,
                               max_depth=10)

start_time = time.time()                       
forest.fit(X_train, y_train)
how_long = time.time() - start_time

y_test_preds = forest.predict(X_test)
mse = mean_squared_error(y_test, y_test_preds)

result = [mse, rmse]
rmse = sqrt(mse)
print(f'RMSE {rmse}')

RMSE 83.31994405896098


In [108]:
sort = forest.feature_importances_.argsort()
fig = px.bar(y=df_list.columns[sort], x=forest.feature_importances_[sort], height=600, title='Wykres miary ważności atrybutów')
fig.update_layout(xaxis_title='wartość',yaxis_title='atrybuty')
fig.show()

In [109]:
list_with_results = ['Las losowy', how_long, result[0], result[1]]
df_compare = df_compare.append(pd.DataFrame([list_with_results], columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE']), ignore_index=True)

C:\Users\manar\AppData\Local\Temp\ipykernel_21800\1972559477.py:2: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [39]:
import keras_tuner
from keras import optimizers
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience= 2)

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2,6)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        if hp.Boolean('dropout_' + str(i)):
            model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(1))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=optimizers.RMSprop(learning_rate=learning_rate),
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    return model
    

In [40]:
import keras_tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=5,
    executions_per_trial=2,
    directory="my_dir",
    project_name="WUM",
)

tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project my_dir\WUM\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\WUM\tuner0.json
Search space summary
Default search space size: 12
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_0 (Boolean)
{'default': False, 'conditions': []}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_1 (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_2 (Boolean)
{'default': False, 'conditions': []}
units_3 (Int)
{'default': None, 'conditions': [], 'min_valu

In [112]:
tuner.search(X_train_part, y_train_part, epochs=15, validation_data=(val_X, val_y), callbacks=[callback])

Trial 5 Complete [00h 01m 41s]
val_loss: 8249.73095703125

Best val_loss So Far: 7039.23828125
Total elapsed time: 00h 06m 51s
INFO:tensorflow:Oracle triggered exit


In [113]:
#tuner.results_summary()

In [41]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]

best_model.layers[0].get_config()['units']

416

In [43]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.build(input_shape=(None, df_list.shape[1]))

In [115]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               54912     
                                                                 
 dense_1 (Dense)             (None, 384)               49536     
                                                                 
 dense_2 (Dense)             (None, 416)               160160    
                                                                 
 dense_3 (Dense)             (None, 1)                 417       
                                                                 
Total params: 265,025
Trainable params: 265,025
Non-trainable params: 0
_________________________________________________________________


In [116]:
model = keras.models.clone_model(best_model, input_tensors=None, clone_function=None)
model.compile(
        optimizer='rmsprop',
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
history = model.fit(X_train_part, y_train_part, epochs = 20,  validation_data= (val_X, val_y), callbacks=[callback])

Epoch 1/20
505/505 [==============================] - 6s 10ms/step - loss: 12216.3076 - root_mean_squared_error: 110.5274 - val_loss: 9609.6445 - val_root_mean_squared_error: 98.0288
Epoch 2/20
505/505 [==============================] - 6s 11ms/step - loss: 8822.1426 - root_mean_squared_error: 93.9263 - val_loss: 7916.3545 - val_root_mean_squared_error: 88.9739
Epoch 3/20
505/505 [==============================] - 4s 8ms/step - loss: 8160.6494 - root_mean_squared_error: 90.3363 - val_loss: 8138.9277 - val_root_mean_squared_error: 90.2160
Epoch 4/20
505/505 [==============================] - 5s 9ms/step - loss: 7617.1562 - root_mean_squared_error: 87.2763 - val_loss: 6628.9155 - val_root_mean_squared_error: 81.4182
Epoch 5/20
505/505 [==============================] - 5s 9ms/step - loss: 7263.5190 - root_mean_squared_error: 85.2263 - val_loss: 6832.6465 - val_root_mean_squared_error: 82.6598
Epoch 6/20
505/505 [==============================] - 5s 10ms/step - loss: 7005.1836 - root_mean

In [117]:
history = model.history

In [118]:
model = keras.models.clone_model(best_model, input_tensors=None, clone_function=None)
model.compile(
        optimizer='rmsprop',
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )


start_time = time.time()  
history = model.fit(X_train, y_train, epochs = len(history.history['val_loss']))
how_long = time.time() - start_time

Epoch 1/8
589/589 [==============================] - 6s 8ms/step - loss: 12068.1006 - root_mean_squared_error: 109.8549
Epoch 2/8
589/589 [==============================] - 5s 9ms/step - loss: 8669.6357 - root_mean_squared_error: 93.1109
Epoch 3/8
589/589 [==============================] - 6s 10ms/step - loss: 7841.1396 - root_mean_squared_error: 88.5502
Epoch 4/8
589/589 [==============================] - 6s 10ms/step - loss: 7455.7002 - root_mean_squared_error: 86.3464
Epoch 5/8
589/589 [==============================] - 6s 9ms/step - loss: 7166.8696 - root_mean_squared_error: 84.6574
Epoch 6/8
589/589 [==============================] - 5s 8ms/step - loss: 6838.5532 - root_mean_squared_error: 82.6955
Epoch 7/8
589/589 [==============================] - 5s 9ms/step - loss: 6664.3291 - root_mean_squared_error: 81.6353
Epoch 8/8
589/589 [==============================] - 8s 14ms/step - loss: 6561.6758 - root_mean_squared_error: 81.0042


In [119]:
result = model.evaluate(X_test,y_test)

253/253 [==============================] - 2s 6ms/step - loss: 6333.0371 - root_mean_squared_error: 79.5804


In [120]:
list_with_results = ['Sieć neuronowa', how_long, result[0], result[1]]
df_compare = df_compare.append(pd.DataFrame([list_with_results], columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE']), ignore_index=True)

C:\Users\manar\AppData\Local\Temp\ipykernel_21800\2593319397.py:2: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [121]:
predict = model.predict(X_test)

253/253 [==============================] - 3s 10ms/step


In [122]:
predict = [item for sublist in predict for item in sublist]
df_result = pd.DataFrame({'predict':predict, 'y':y_test.tolist()})
df_result = df_result.sort_values(by=['y'])

In [123]:
fig = go.Figure()

fig.add_scatter(y = df_result['predict'], name='predicted value')
fig.add_scatter(y = df_result['y'], name='real value')

In [124]:
y_test_mean = y_test.mean()
y_list_mean = np.full(shape=y_test.shape[0], fill_value=y_test_mean, dtype=np.float32)
mse = mean_squared_error(y_test, y_list_mean)

rmse = sqrt(mse)
result = [mse, rmse]

list_with_results = ['Średnia', 0, result[0], result[1]]
df_compare = df_compare.append(pd.DataFrame([list_with_results], columns=['Nazwa', 'Czas', 'F. Straty', 'RMSE']), ignore_index=True)

C:\Users\manar\AppData\Local\Temp\ipykernel_21800\102260392.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [125]:
fig = px.bar(df_compare, x=df_compare["Nazwa"],y=df_compare["Czas"], title='Wykres czasu dla różnych modeli', labels={'Czas':'Czas [s]','Nazwa':'Model'})
fig.show()

In [126]:
fig = px.bar(df_compare, x=df_compare["Nazwa"],y=df_compare["RMSE"], title='Wykres RMSE dla różnych modeli', labels={'y':'Wartość RMSE [%]','Nazwa':'Model'})
fig.show()

In [127]:
df_compare

,Nazwa,Czas,F. Straty,RMSE
0,Lasso,6.875454,6414.940224,80.093322
1,Las losowy,16.802703,6942.213078,80.093322
2,Sieć neuronowa,46.527987,6333.037109,79.580383
3,Średnia,0.000000,16977.474609,130.297639
